# Hyperparameter Tuning Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

# Gridsearch Hyperparameters

In the guided project, you learned how to use sklearn's `GridsearchCV` and `keras-tuner` libraries to tune the hyperparameters of a neural network model. For your module project you'll continue using these two libraries, however we are going to make things a little more interesting for you.

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class.



**Don't forget to switch to GPU on Colab!**

In [1]:
pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [2]:
pip install scikeras[tensorflow]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 56.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      

## 0.1 Imports and installs

In [3]:
# native python libraries imports
import math
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# sklearn imports
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# keras imports
import keras

from keras import Sequential
from keras.layers import Dense
from keras.activations import relu, sigmoid
from keras.optimizers import Adam, SGD
from keras.utils import get_file

# required for compatibility between sklearn and keras
from scikeras.wrappers import KerasClassifier, KerasRegressor

# install keras-tuner
from keras_tuner import RandomSearch, BayesianOptimization
from keras_tuner import HyperParameters

## 0.2 Load quickdraw data

In [4]:
def load_quickdraw10():
    """
    Fill out this doc string, and comment the code, for practice in writing the kind of code that will get you hired.
    """

    URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"

    path_to_zip = get_file('./quickdraw10.npz', origin=URL_, extract=False)

    data = np.load(path_to_zip)

    # normalize your image data
    max_pixel_value = 255
    X = data['arr_0']/max_pixel_value
    Y = data['arr_1']

    return train_test_split(X, Y, shuffle=True)

In [5]:
X_train, X_test, y_train, y_test = load_quickdraw10()

25421363/25421363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
X_train.shape

(75000, 784)

In [7]:
y_train.shape

(75000,)

_____

# Experiment 1

## Tune Hyperperameters using Enhanced GridsearchCV

We are going to use GridsearchCV again to tune a deep learning model however we are going to add some additional functionality to our gridsearch.

Specifically, we are going to automate the generation of how many nodes to use in a layer and how many layers to use in a model!

By the way, yes, there is a function within a function. Try to not let that bother you. An alternative to this would be to create a class. If you're up for the challenge give it a shot. However, consider this a stretch goal that you come back to after you finish going through this assignment.


### Objective

The objective of this experiment is to show you how to automate the generation of layers and layer nodes for the purposes of gridsearch. <br>
Up until now, we've been manually selecting the number of layers and layer nodes.

In [8]:
# Function to create model, required for KerasClassifier
def create_model(n_layers,  first_layer_nodes, last_layer_nodes, act_funct ="relu", negative_node_incrementation=True):
    """"
    Returns a compiled keras model

    Parameters
    ----------
    n_layers: int
        number of hidden layers in model
        To be clear, this excludes the input and output layer.

    first_layer_nodes: int
        Number of nodes in the first hidden layer

    last_layer_nodes: int
        Number of nodes in the last hidden layer (this is the layer just prior to the output layer)

     act_funct: string
         Name of activation function to use in hidden layers (this excludes the output layer)

    Returns
    -------
    model: keras object
    """

    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer.
        To be clear, this excludes the input and output layer.

        Note
        ----
        Number of nodes in each layer is linearly incremented.
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer
        """

        # throws an error if n_layers is less than 2
        assert n_layers >= 2, "n_layers must be 2 or greater"

        layers = []

        # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
        # THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
        # when set to True number of nodes are decreased for subsequent layers
        # NOTE: the order of the number of nodes doesn't matter
        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to increment towards smaller numbers
            nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)

        # when set to False number of nodes are increased for subsequent layers
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers
            nodes_increment = (last_layer_nodes + first_layer_nodes)/ (n_layers-1)

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer
            nodes = nodes + nodes_increment

        return layers

    # create model
    model = Sequential()

    n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)

    for i in range(1, n_layers+1):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=act_funct))
        else:
            model.add(Dense(n_nodes[i-1], activation=act_funct))


    # output layer
    model.add(Dense(10, # 10 unit/neurons in output layer because we have 10 possible labels to predict
                    activation='softmax')) # use softmax for a label set greater than 2

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam', # adam is a good default optimizer
                  metrics=['accuracy'])

    # do not include model.fit() inside the create_model function
    # KerasClassifier is expecting a complied model
    return model


## 1.1 Explore `create_model`



The helper function `gen_layer_nodes()` which is contained inside `create_model()` <br>
returns a list containing the number of nodes for each successive layer.<br>

Let's check that `gen_layer_nodes()` behaves as expected. <br>
In other words, we'll perform a **Unit Test!**

In [9]:
def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
    """
    Generates and returns the number of nodes in each hidden layer.
    To be clear, this excludes the input and output layer.

    Note
    ----
    Number of nodes in each layer is linearly incremented.
    For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

    Parameters
    ----------
    n_layers: int
        Number of hidden layers
        This values should be 2 or greater

    first_layer_nodes: int

    last_layer_nodes: int

    Returns
    -------
    layers: list of ints
        Contains number of nodes for each layer
    """

    # throws an error if n_layers is less than 2
    assert n_layers >= 2, "n_layers must be 2 or greater"

    layers = []

    # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
    # THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
    # when set to True number of nodes are decreased for subsequent layers
    # NOTE: the order of the number of nodes doesn't matter
    if negative_node_incrementation:
        # subtract this amount from previous layer's nodes in order to increment towards smaller numbers
        nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
        #print(f'nodes increment = {nodes_increment}')

    # when set to False number of nodes are increased for subsequent layers
    else:
        # add this amount from previous layer's nodes in order to increment towards larger numbers
        nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
        #print(f'nodes increment = {nodes_increment}')

    nodes = first_layer_nodes

    for i in range(1, n_layers+1):

        layers.append(math.ceil(nodes))

        # increment nodes for next layer
        nodes = nodes + nodes_increment

    return layers


### `negative_node_incrementation = True`
For this case we want the number of nodes to _decrease_ by a constant number for successive layers. <br>So `first_layer_nodes` must be _larger_ than `last_layer_nodes`

In [10]:
n_layers = 5
first_layer_nodes = 500
last_layer_nodes = 100
negative_node_incrementation = True
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [500, 400, 300, 200, 100]


### `negative_node_incrementation = False`
For this case we want the number of nodes to _increase_ by a constant number for successive layers. <br>So `first_layer_nodes` must be _smaller_ than `last_layer_nodes`

In [11]:
n_layers = 5
first_layer_nodes = 100
last_layer_nodes = 500
negative_node_incrementation = False
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [100, 200, 300, 400, 500]


### OK, the Unit Test is passed!

### Let's build a few models<br>
in order to understand how `create_model()` works in practice.

### Build a model, setting `negative_node_incrementation = True`

Use `create_model` to build a model.

- Set `n_layers = 10`
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`


In [12]:
# use create_model to create a model

# YOUR CODE HERE
model = create_model(n_layers=10,
                     first_layer_nodes=500,
                     last_layer_nodes=100,
                     act_funct="relu",
                     negative_node_incrementation=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# run model.summary() and make sure that you understand the model architecture that you just built
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values.
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 500)                 │         392,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 456)                 │         228,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 412)                 │         188,284 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 367)                 │         151,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 323)                 │         118,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 278)                 │          90,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 234)                 │          65,286 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 189)                 │          44,415 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 145)                 │          27,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 100)                 │          14,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,322,608 (5.05 MB)

 Trainable params: 1,322,608 (5.05 MB)

 Non-trainable params: 0 (0.00 B)

### Build a model, setting `negative_node_incrementation = False`

Use `create_model` to build a model.

- Set `n_layers = 10`
- Set `first_layer_nodes = 100`
- Set `last_layer_nodes = 500`
- Set `act_funct = "relu"`

In [14]:
# use create_model to create a model

# YOUR CODE HERE
model = create_model(n_layers=10,
                     first_layer_nodes=500,
                     last_layer_nodes=100,
                     act_funct="relu",
                     negative_node_incrementation=False)

In [15]:
# run model.summary() and make sure that you understand the model architecture that you just built
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values.
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                     │ (None, 500)                 │         392,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 567)                 │         284,067 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 634)                 │         360,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 700)                 │         444,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 767)                 │         537,667 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 834)                 │         640,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 900)                 │         751,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 967)                 │         871,267 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1034)                │       1,000,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 1100)                │       1,138,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 10)                  │          11,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,432,547 (24.54 MB)

 Trainable params: 6,432,547 (24.54 MB)

 Non-trainable params: 0 (0.00 B)

## 1.2 Create a grid search using `sklearn`

### Hyperparameter search

In [16]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3],
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [18]:
model = KerasClassifier(create_model)

In [19]:
%%time
# Create Grid Search
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-2,
                    verbose=1,
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


ValueError: Invalid parameter first_layer_nodes for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(first_layer_nodes=500)`
Check the list of available parameters with `estimator.get_params().keys()`

In [20]:
best_model = grid_result.best_estimator_

NameError: name 'grid_result' is not defined

In [ ]:
best_model.get_params()

Ok, now that we've played around a bit with  `create_model`, let's build a  simpler model that we'll use to run gridsearches.

### Build model

Use `create_model` to build a model.

- Set `n_layers = 2`
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`
- Make sure that `negative_node_incrementation = True`

In [ ]:
# use create_model to create a model

###BEGIN SOLUTION
# use create_model to create a model

###END SOLUTION

In [ ]:
# run model.summary() and make sure that you understand the model architecture that you just built
model.summary()

In [ ]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3],
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [ ]:
model = KerasClassifier(create_model)

In [ ]:
%%time
# Create Grid Search
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-2,
                    verbose=1,
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [ ]:
best_model = grid_result.best_estimator_

In [ ]:
best_model.get_params()

-----

# Experiment 2: Run the Gridsearch Algorithms

In this section, we are going to use the same model and dataset in order to benchmark 3 different gridsearch approaches:

- Gridsearch
- Random Search
- Bayesian Optimization.


Our goal in this experiment is two-fold. We want to see which appraoch

- Scores the highest accuracy
- Has the shortest run time

We want to see how these 3 gridsearch approaches handle these trade-offs and to give you a sense of those trades offs.

### Trade-offs

`Gridsearch` will train a model on every single unique hyperparameter combination, this guarantees that you'll get the highest possible accuracy from your parameter set but your gridsearch might have a very long run-time.

`Random Search` will randomly sample from your parameter set which, depending on how many samples, the run-time might be significantly cut down but you might or might not sample the parameters that correspond to the heightest possible accuracies.

`Bayesian Optimization` has a bit of intelligence built into it's search algorithm but you do need to manually select some parameters which may greatly influence the model learning outcomes.

-------
### Build our hyperparameter dictionary

In [ ]:
# build out our hyperparameter dictionary
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate',values=[1e-1, 1e-2, 1e-3])
hp.Choice('activation',values=["relu", "sigmoid"])

---------
## 2.1 Gridsearch Optimization


### Populate a `sklearn` compatible parameter dictionary

In [ ]:
# build out our hyperparameter dictionary
hyper_parameters = {
    # BUG Fix: cast array as list otherwise GridSearchCV will throw error
    "units": np.arange(32, 512, 32).tolist(),
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "activation":["relu", "sigmoid"]
}

In [ ]:
hyper_parameters

### Build a `sklearn` compatible model function

In [ ]:
def build_model(units, learning_rate, activation):
    """
    Returns a complie keras model ready for keras-tuner gridsearch algorithms
    """

    model = Sequential()

    # hidden layer
    model.add(Dense(units, activation=activation))

    # output layer
    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

### Apply the "wrapper" to make the model compatible with `sklearn`

In [ ]:
model = KerasClassifier(model=build_model)

In [ ]:
# save start time
start = time()

# Create Grid Search
grid = GridSearchCV(estimator=model,
                    param_grid=hyper_parameters,
                    n_jobs=-2,
                    verbose=1,
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# save end time
end = time()

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [ ]:
# total run time
total_run_time_in_miniutes = (end - start)/60
total_run_time_in_miniutes

In [ ]:
grid_result.best_params_

In [ ]:
# because all other optimization approaches are reporting test set score
# let's calculate the test set score in this case
best_model = grid_result.best_estimator_
test_acc = best_model.score(X_test, y_test)

In [ ]:
test_acc

 ### Results

Identify and write the the best performing hyperparameter combination and model score.



YOUR ANSWER HERE

------
## 2.2 Random Search with `keras-tuner`

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `RandomSearch` tuner.

### Build model
---

In [ ]:
# because gridsearching can take a lot of time and we are bench marking 3 different approaches
# let's build a simple model to minimize run time

def build_model(hp):
    """
    Returns a complied keras model ready for keras-tuner gridsearch algorithms
    """

    model = Sequential()

    # hidden layer
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))

    # output layer
    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

In [ ]:
# how many unique hyperparameter combinations do we have?
# HINT: take the product of the number of possible values for each hyperparameter
# save your answer to n_unique_hparam_combos

# YOUR CODE HERE

n_values_in_units = 10
n_values_in_learning_rate = 3
n_values_in_activation = 2

n_unique_hparam_combos = n_values_in_units * n_values_in_learning_rate * n_values_in_activation

In [ ]:
# how many of these do we want to randomly sample?
# let's pick 25% of n_unique_hparam_combos param combos to sample
# save this number to n_param_combos_to_sample

# YOUR CODE HERE
fraction_to_sample = 0.25
n_param_combos_to_sample = int(fraction_to_sample * n_unique_hparam_combos)

### Instantiate a `RandomSearch()` object for your grid search

In [ ]:
random_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=n_param_combos_to_sample, # number of times to sample the parameter set and build a model
    seed=1234,
    hyperparameters=hp, # pass in our hyperparameter dictionary
    directory='./keras-tuner-trial',
    project_name='random_search'
)

In [ ]:
 # take note of Total elapsed time in print out -- took ~10 minutes without GPU
random_tuner.search(
    X_train,
    y_train,
    epochs=3,
    validation_data=(X_test, y_test),
)

In [ ]:
# identify the best score and hyperparamter (should be at the top since scores are ranked)
random_tuner.results_summary()

 ### Results

Identify and write the the best performing hyperparameter combination and model score.
Note that because this is Random Search, multiple runs might have slighly different outcomes.



YOUR ANSWER HERE

------
## 2.3 Bayesian Optimization with `keras-tuner`

![](https://upload.wikimedia.org/wikipedia/commons/0/02/GpParBayesAnimationSmall.gif)

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `BayesianOptimization` tuner.

Pay special attention to these `BayesianOptimization` parameters: `num_initial_points` and `beta`.

`num_initial_points`:

Number of randomly selected hyperparameter combinations to try before applying bayesian probability to determine liklihood of which param combo to try next based on expected improvement


`beta`:

Larger values means more willing to explore new hyperparameter combinations (analogous to searching for the global minimum in Gradient Descent), smaller values means that it is less willing to try new hyperparameter combinations (analogous to getting stuck in a local minimum in Gradient Descent).

As a start, error on the side of larger values. What defines a small or large value you ask? That question would pull us into the mathematical intricacies of Bayesian Optimization and Gaussian Processes. For simplicity, notice that the default value is 2.6 and work from there.

In [ ]:
# we know that 24 samples is about 25% of 96 possible hyper-parameter combos
# let's set up a run with the same parameters we used for RandomSearch() so the comparison will be aplles-to-apples
# feel free to play with any of these numbers later
max_trials=24
num_initial_points=5
beta=5.0

#### Instantiate a `BayesianOptimization()` object for your grid search

In [ ]:
bayesian_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=max_trials,
    hyperparameters=hp, # pass in our hyperparameter dictionary
    num_initial_points=num_initial_points,
    beta=beta,
    seed=1234,
    directory='./keras-tuner-trial',
    project_name='bayesian_optimization_4',
)

In [ ]:
bayesian_tuner.search(
    X_train,
    y_train,
    epochs=3,
    validation_data=(X_test, y_test),
)

In [ ]:
bayesian_tuner.results_summary()

 ### Results

Identify and write the the best performing hyperparameter combination and model score.
Note that because this is  Bayesian Optimization, multiple runs might have slighly different outcomes.



YOUR ANSWER HERE

We should point out that Gridsearch split the training set internally and created a test set whereas keras-tuner allows us to pass in a test set. This means that the keras-tuner algorithms were using one test set and our skearn GridSearchCV was using a different test set - so this isn't a perfectly exact 1-to-1 comparision but it'll have to do. In order to compensate for this, we did score the best model on the same test set that keras-tuner used.

_______

# Conclusion

The spirit of this experiment is to expose you to the idea of benchmarking and comparing the trade-offs of various gridsearch approaches.

Even if we did find a way to pass in the original test set into GridSearchCV, we can see that both Random Search and Bayesian Optimization are arguably better alternatives to a brute force grid search when we consider the trade-offs of run time and locating the best performing model.

----

# Stretch Goals

- Feel free to run whatever gridsearch experiments on whatever models you like!

In [ ]:
# this is your open playground - be free to explore as you wish
